In [10]:
import csv
import pandas as pd
from tensorflow import keras
from sklearn.model_selection import train_test_split
from utils import *
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1' 
import tensorflow as tf
import tensorflowjs as tfjs
from preprocess import Preprocessor
tfjs_model_dir = 'model'

In [2]:
def make_df(folder, subfolder):
    images_in_folder=os.path.join(folder, subfolder)
    train_preprocessor = Preprocessor(
    images_in_folder,)
    dataframe =train_preprocessor.process()   
    return dataframe  

# loading final csv file
def load_data(df):
    df.drop(['filename'],axis=1, inplace=True)
    classes = df.pop('class_name').unique()
    y = df.pop('class_no')
    X = df.astype('float64')
    y = keras.utils.to_categorical(y)
    return X, y, classes

In [3]:
train= make_df("dataset", "train")
test= make_df("dataset", "test")
test.head

100%|██████████| 324/324 [00:02<00:00, 113.07it/s]


['Skipped 0.jpg:Zone.Identifier - Invalid file type', 'Skipped 1.jpg:Zone.Identifier - Invalid file type', 'Skipped 10.jpg:Zone.Identifier - Invalid file type', 'Skipped 11.jpg:Zone.Identifier - Invalid file type', 'Skipped 12.jpg:Zone.Identifier - Invalid file type', 'Skipped 13.jpg:Zone.Identifier - Invalid file type', 'Skipped 14.jpg:Zone.Identifier - Invalid file type', 'Skipped 15.jpg:Zone.Identifier - Invalid file type', 'Skipped 16.jpg:Zone.Identifier - Invalid file type', 'Skipped 17.jpg:Zone.Identifier - Invalid file type', 'Skipped 18.jpg:Zone.Identifier - Invalid file type', 'Skipped 19.jpg:Zone.Identifier - Invalid file type', 'Skipped 2.jpg:Zone.Identifier - Invalid file type', 'Skipped 20.jpg:Zone.Identifier - Invalid file type', 'Skipped 21.jpg:Zone.Identifier - Invalid file type', 'Skipped 22.jpg:Zone.Identifier - Invalid file type', 'Skipped 23.jpg:Zone.Identifier - Invalid file type', 'Skipped 24.jpg:Zone.Identifier - Invalid file type', 'Skipped 25.jpg:Zone.Identifie

100%|██████████| 166/166 [00:01<00:00, 114.36it/s]

['Skipped 0.jpg:Zone.Identifier - Invalid file type', 'Skipped 1.jpg:Zone.Identifier - Invalid file type', 'Skipped 10.jpg:Zone.Identifier - Invalid file type', 'Skipped 100.jpg:Zone.Identifier - Invalid file type', 'Skipped 101.jpg:Zone.Identifier - Invalid file type', 'Skipped 102.jpg:Zone.Identifier - Invalid file type', 'Skipped 103.jpg:Zone.Identifier - Invalid file type', 'Skipped 104.jpg:Zone.Identifier - Invalid file type', 'Skipped 105.jpg:Zone.Identifier - Invalid file type', 'Skipped 106.jpg:Zone.Identifier - Invalid file type', 'Skipped 107.jpg:Zone.Identifier - Invalid file type', 'Skipped dataset/test/cheat/108.jpg - No valid pose detected', 'Skipped 108.jpg:Zone.Identifier - Invalid file type', 'Skipped dataset/test/cheat/109.jpg - No valid pose detected', 'Skipped 109.jpg:Zone.Identifier - Invalid file type', 'Skipped 11.jpg:Zone.Identifier - Invalid file type', 'Skipped 110.jpg:Zone.Identifier - Invalid file type', 'Skipped 111.jpg:Zone.Identifier - Invalid file type',

<bound method NDFrame.head of     filename  NOSE_x  NOSE_y  NOSE_score  LEFT_EYE_x  LEFT_EYE_y  \
0      0.jpg    61.0    58.0    0.536201        56.0        54.0   
1      1.jpg    60.0    54.0    0.583177        54.0        51.0   
2     10.jpg    98.0    55.0    0.588507        93.0        50.0   
3    100.jpg   146.0    61.0    0.435530       142.0        56.0   
4    101.jpg   149.0    58.0    0.612336       144.0        54.0   
..       ...     ...     ...         ...         ...         ...   
239    8.jpg    95.0    97.0    0.538388        97.0        95.0   
240   80.jpg   109.0     5.0    0.534450       109.0         1.0   
241   81.jpg   101.0     3.0    0.553880       103.0         0.0   
242   82.jpg   100.0     3.0    0.629847       102.0         0.0   
243    9.jpg    95.0    97.0    0.414240        97.0        94.0   

     LEFT_EYE_score  RIGHT_EYE_x  RIGHT_EYE_y  RIGHT_EYE_score  ...  \
0          0.577895         61.0         54.0         0.583147  ...   
1          

In [4]:
train.head

<bound method NDFrame.head of     filename  NOSE_x  NOSE_y  NOSE_score  LEFT_EYE_x  LEFT_EYE_y  \
0      0.jpg    61.0    58.0    0.536201        56.0        54.0   
1      1.jpg    60.0    54.0    0.583177        54.0        51.0   
2     10.jpg    98.0    55.0    0.588507        93.0        50.0   
3     11.jpg   114.0    67.0    0.540989       111.0        62.0   
4     12.jpg   118.0    80.0    0.504765       114.0        75.0   
..       ...     ...     ...         ...         ...         ...   
666   95.jpg    63.0    27.0    0.696270        68.0        23.0   
667   96.jpg    68.0    26.0    0.716201        72.0        23.0   
668   97.jpg    74.0    29.0    0.526421        77.0        24.0   
669   98.jpg    81.0    30.0    0.711263        84.0        26.0   
670   99.jpg    86.0    30.0    0.605774        89.0        26.0   

     LEFT_EYE_score  RIGHT_EYE_x  RIGHT_EYE_y  RIGHT_EYE_score  ...  \
0          0.577895         61.0         54.0         0.583147  ...   
1          

In [11]:
X, y, class_names = load_data(train)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.15)
X_test, y_test, _ = load_data(test)

KeyError: "['filename'] not found in axis"

In [6]:
def get_center_point(landmarks, left_bodypart, right_bodypart):
    """Calculates the center point of the two given landmarks."""
    left = tf.gather(landmarks, left_bodypart.value, axis=1)
    right = tf.gather(landmarks, right_bodypart.value, axis=1)
    center = left * 0.5 + right * 0.5
    return center


def get_pose_size(landmarks, torso_size_multiplier=2.5):
    """Calculates pose size.

    It is the maximum of two values:
    * Torso size multiplied by `torso_size_multiplier`
    * Maximum distance from pose center to any pose landmark
    """
    # Hips center
    hips_center = get_center_point(landmarks, BodyPart.LEFT_HIP, 
                                 BodyPart.RIGHT_HIP)

    # Shoulders center
    shoulders_center = get_center_point(landmarks, BodyPart.LEFT_SHOULDER,
                                      BodyPart.RIGHT_SHOULDER)

    # Torso size as the minimum body size
    torso_size = tf.linalg.norm(shoulders_center - hips_center)
    # Pose center
    pose_center_new = get_center_point(landmarks, BodyPart.LEFT_HIP, 
                                     BodyPart.RIGHT_HIP)
    pose_center_new = tf.expand_dims(pose_center_new, axis=1)
    # Broadcast the pose center to the same size as the landmark vector to
    # perform substraction
    pose_center_new = tf.broadcast_to(pose_center_new,
                                    [tf.size(landmarks) // (17*2), 17, 2])

    # Dist to pose center
    d = tf.gather(landmarks - pose_center_new, 0, axis=0,
                name="dist_to_pose_center")
    # Max dist to pose center
    max_dist = tf.reduce_max(tf.linalg.norm(d, axis=0))

    # Normalize scale
    pose_size = tf.maximum(torso_size * torso_size_multiplier, max_dist)
    return pose_size



def normalize_pose_landmarks(landmarks):
    """Normalizes the landmarks translation by moving the pose center to (0,0) and
    scaling it to a constant pose size.
  """
  # Move landmarks so that the pose center becomes (0,0)
    pose_center = get_center_point(landmarks, BodyPart.LEFT_HIP, 
                                 BodyPart.RIGHT_HIP)

    pose_center = tf.expand_dims(pose_center, axis=1)
    # Broadcast the pose center to the same size as the landmark vector to perform
    # substraction
    pose_center = tf.broadcast_to(pose_center, 
                                [tf.size(landmarks) // (17*2), 17, 2])
    landmarks = landmarks - pose_center

    # Scale the landmarks to a constant pose size
    pose_size = get_pose_size(landmarks)
    landmarks /= pose_size
    return landmarks


def landmarks_to_embedding(landmarks_and_scores):
    """Converts the input landmarks into a pose embedding."""
    # Reshape the flat input into a matrix with shape=(17, 3)
    reshaped_inputs = keras.layers.Reshape((17, 3))(landmarks_and_scores)

    # Normalize landmarks 2D
    landmarks = normalize_pose_landmarks(reshaped_inputs[:, :, :2])
    # Flatten the normalized landmark coordinates into a vector
    embedding = keras.layers.Flatten()(landmarks)
    return embedding


def preprocess_data(X_train):
    processed_X_train = []
    for i in range(X_train.shape[0]):
        embedding = landmarks_to_embedding(tf.reshape(tf.convert_to_tensor(X_train.iloc[i]), (1, 51)))
        processed_X_train.append(tf.reshape(embedding, (34)))
    return tf.convert_to_tensor(processed_X_train)


In [7]:
processed_X_train = preprocess_data(X_train)
processed_X_val =  preprocess_data(X_val)
processed_X_test = preprocess_data(X_test)

inputs = tf.keras.Input(shape=(34,))
layer = keras.layers.Dense(128, activation=tf.nn.relu6)(inputs)
layer = keras.layers.Dropout(0.5)(layer)
layer = keras.layers.Dense(64, activation=tf.nn.relu6)(layer)
layer = keras.layers.Dropout(0.5)(layer)
outputs = keras.layers.Dense(len(class_names), activation="softmax")(layer)


In [8]:
model = keras.Model(inputs, outputs)
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Add a checkpoint callback to store the checkpoint that has the highest
# validation accuracy.
# Update checkpoint path to use the .keras extension
checkpoint_path = "weights.best.keras"

checkpoint = keras.callbacks.ModelCheckpoint(checkpoint_path,
                                             monitor='val_accuracy',
                                             verbose=1,
                                             save_best_only=True,
                                             mode='max')

earlystopping = keras.callbacks.EarlyStopping(monitor='val_accuracy', 
                                              patience=20)


In [9]:
# Start training
print('--------------TRAINING----------------')
history = model.fit(processed_X_train, y_train,
                    epochs=200,
                    batch_size=16,
                    validation_data=(processed_X_val, y_val),
                    callbacks=[checkpoint, earlystopping])


print('-----------------EVAUATION----------------')
loss, accuracy = model.evaluate(processed_X_test, y_test)
print('LOSS: ', loss)
print("ACCURACY: ", accuracy)


tfjs.converters.save_keras_model(model, tfjs_model_dir)
print('tfjs model saved at ',tfjs_model_dir)


--------------TRAINING----------------
Epoch 1/200
19/36 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4617 - loss: 1.0513   
Epoch 1: val_accuracy improved from -inf to 0.52475, saving model to weights.best.keras
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.4930 - loss: 1.0179 - val_accuracy: 0.5248 - val_loss: 0.8258
Epoch 2/200
22/36 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6440 - loss: 0.7816 
Epoch 2: val_accuracy improved from 0.52475 to 0.79208, saving model to weights.best.keras
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6593 - loss: 0.7575 - val_accuracy: 0.7921 - val_loss: 0.5853
Epoch 3/200
26/36 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8282 - loss: 0.5580 
Epoch 3: val_accuracy improved from 0.79208 to 0.93069, saving model to weights.best.keras
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8378 - loss: 0.5434 - val_accuracy: 0.9307 - val_loss: 0.3867
Epoch 4/200
34/36 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9124 - loss: 0.3748 


LOSS:  0.014045845717191696
ACCURACY:  0.9959016442298889
failed to lookup keras version from the file,
    this is likely a weight only file
tfjs model saved at  model
